In [ ]:
import os
import re
import pandas as pd
import numpy as np

In [ ]:
def change_dtype_ser(ser):
    
    if ser.dtype == int:
        return ser.astype(np.int32)
    
    if ser.dtype == float:
        return ser.astype(np.float32)
    
    if ser.dtype == np.object:
        return ser.astype("category")
    
    return ser
    

def change_dtype_df(df):
    """
    change types of columns to reduce memory size
    :param df: dataframe
    :return df: dataframe
    """
    df["fecha_dato"] = pd.to_datetime(df["fecha_dato"])
    df["fecha_alta"] = pd.to_datetime(df["fecha_alta"])
    
    memory = df.memory_usage().sum() / 10**6
    print("Memory usage before changing types %0.2f MB" % memory)

    for col in df.columns:
        df[col] = change_dtype_ser(df[col])

    memory = df.memory_usage().sum() / 10 ** 6
    print("Memory usage after changing types %0.2f MB" % memory)
    return df


def load_csv(filename):
    df = pd.read_csv(filename)
    df = change_dtype_df(df)
    return df

In [ ]:
INP_DIR = "data/data_"
OUT_DIR1 = "data/data1_"
OUT_DIR2 = "data/data2_"

In [ ]:
df_train = load_csv(os.path.join(INP_DIR, "train_cleaned.csv"))
df_test = load_csv(os.path.join(INP_DIR, "test_cleaned.csv"))

In [ ]:
# drop this column becuase it is too imbalanced
df_train = df_train.drop(["ind_empleado"], axis=1)
df_test = df_test.drop(["ind_empleado"], axis=1)

In [ ]:
# days from when the data is recorded 
df_train["fecha_alta"] = (df_train["fecha_alta"] - df_train["fecha_dato"]).dt.days
df_test["fecha_alta"] = (df_test["fecha_alta"] - df_test["fecha_dato"]).dt.days

In [ ]:
df_train.isnull().sum().sum(), df_test.isnull().sum().sum()

In [ ]:
df_train.shape, df_test.shape

In [ ]:
df_train.columns

In [ ]:
df_test.columns

In [ ]:
PROD_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1$", col)]
print(PROD_COLS)

In [ ]:
NEW_PURCH_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1_NEW_PUR$", col)]
print(NEW_PURCH_COLS, "\n")

prod_popul = df_train[NEW_PURCH_COLS].sum(axis=0)
prod_popul = prod_popul.sort_values(ascending=False)/prod_popul.sum() * 100
print(prod_popul, "\n")

NEW_PURCH_COLS = prod_popul.sort_values(ascending=False).index.to_list()
print(NEW_PURCH_COLS)

In [ ]:
PURCH_CANCEL_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1_PUR_OR_CANCEL$", col)]
print(PURCH_CANCEL_COLS)

In [ ]:
# total number of prod
df_train["TOTAL_PRODS"] = df_train[PROD_COLS].sum(axis=1)

In [ ]:
DEMOG_COLS = [col for col in df_train.columns 
    if col not in PROD_COLS + NEW_PURCH_COLS + PURCH_CANCEL_COLS + ["fecha_dato", "ncodpers", "TOTAL_PRODS"]]
print(DEMOG_COLS)

In [ ]:
LAG_COLS = ["fecha_alta", "ind_nuevo", "antiguedad", "indrel", "tiprel_1mes", 
            "ind_actividad_cliente", "renta", "segmento", "TOTAL_PRODS"]

In [ ]:
def _is_unique(x):
    return len(x) == x.nunique()


def _onehot_reduce(df):
    df = df.copy()
    values = df.values
    
    new_values = values.copy()
    for i in range(values.shape[0]):
        row = values[i]
        if row.sum() > 1:
            new_row = np.zeros([row.shape[0]], dtype=np.int)
            
            where_1 = np.where(row == 1)[0]
            np.random.shuffle(where_1)
            idx = where_1[0]
            new_row[idx] = 1
            
            new_values[i] = new_row
    
    new_df = pd.DataFrame(data=new_values, index=df.index, columns=df.columns)
    return new_df


def extract_subset(df, row_filter, cols):
    df = df.copy()
    return df.loc[row_filter, cols]



def extract_y(df, timestamp, y_cols=NEW_PURCH_COLS):
    # only use row when customer buys at least one product
    any_new_pur = df[y_cols].sum(axis=1) > 0
    row_filter = (df["fecha_dato"] == timestamp) & any_new_pur
    cols = ["ncodpers"] + y_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"
    
    df_out.columns = [col.replace("_NEW_PUR", "") if col.endswith("_NEW_PUR") else col for col in df_out.columns]
    
    id_col = df_out["ncodpers"]
    pur_cols = [col for col in df_out.columns if col != "ncodpers"]
    df_out = _onehot_reduce(df_out[pur_cols])
    df_out["ncodpers"] = id_col
    
    return df_out


def extract_x_demog(df, timestamp, customer_ids, demog_cols=DEMOG_COLS):
    row_filter = (df["fecha_dato"] == timestamp) & df["ncodpers"].isin(customer_ids)
    cols = ["ncodpers"] + demog_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"

    return df_out


# This function can give dataframe having less rows than len(customer_ids)
# This is becuse number of row at lag_timestamp is not the same as at timestamp
def extract_lag_features(df, timestamp_lag, customer_ids, suffix="_LAG", 
                         sel_cols=PROD_COLS+PURCH_CANCEL_COLS+LAG_COLS):
    row_filter = (df["fecha_dato"] == timestamp_lag) & df["ncodpers"].isin(customer_ids)
    cols = ["ncodpers"] + sel_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"
    
    df_out.columns = ["ncodpers"] + [col + suffix for col in df_out.columns if col != "ncodpers"]
    
    return df_out



def extract_X_y_train(df, timestamp, timestamp_lags):
    y_train = extract_y(df, timestamp)
    print("y_train.shape", y_train.shape)
    
    x_demog = extract_x_demog(df, timestamp, customer_ids=y_train["ncodpers"])
    print("x_demog.shape:", x_demog.shape)
    assert y_train.shape[0] == x_demog.shape[0], "x_demog must have the same number of rows as y_train"
    
    x_lags = []
    for t, lag in enumerate(timestamp_lags):
        assert pd.to_datetime(lag) < pd.to_datetime(timestamp), lag + " lag is not before timestamp " +  timestamp
        lag_label = "_LAG%d" % (t + 1)
        
        x_lag = extract_lag_features(df, lag, customer_ids=y_train["ncodpers"], suffix=lag_label)
        print(lag, lag_label, x_lag.shape)
        x_lags.append(x_lag)
    
    X_train = y_train[["ncodpers"]].merge(x_demog, how="left", on="ncodpers")
    print("Nulls after merging y and x_demog:", X_train.isnull().sum().sum())
    
    for t, x_lag in enumerate(x_lags):
        X_train = X_train.merge(x_lag, how="left", on="ncodpers")
        print("Nulls at %d:" %(t + 1), X_train.isnull().sum().sum())
    
    print("X_train.shape:", X_train.shape)
    print("y_train.shape:", y_train.shape)
    return X_train, y_train


def extract_X_test(train, test, timestamp, timestamp_lags):
    x_demog = extract_x_demog(test, timestamp, customer_ids=test["ncodpers"])
    print("x_demog.shape:", x_demog.shape)
    print("Nulls of x_demog:", x_demog.isnull().sum().sum())
    
    x_lags = []
    for t, lag in enumerate(timestamp_lags):
        assert pd.to_datetime(lag) < pd.to_datetime(timestamp), lag + " lag is not before timestamp " +  timestamp
        lag_label = "_LAG%d" % (t + 1)
        
        x_lag = extract_lag_features(train, lag, customer_ids=test["ncodpers"], suffix=lag_label)
        print(lag, lag_label, x_lag.shape)
        x_lags.append(x_lag)
        
    X_test = x_demog
    for t, x_lag in enumerate(x_lags):
        X_test = X_test.merge(x_lag, how="left", on="ncodpers")
        print("Nulls at %d:" %(t + 1), X_test.isnull().sum().sum())
    
    print("X_test.shape:", X_test.shape)
    return X_test

In [ ]:
def extract_x_demog(df, timestamp, customer_ids, demog_cols=DEMOG_COLS):
    row_filter = (df["fecha_dato"] == timestamp) & df["ncodpers"].isin(customer_ids)
    cols = ["ncodpers"] + demog_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"

    return df_out

In [ ]:
df_train["fecha_dato"].unique()

# Use 6 month lags

## Extract `X_2016_04`, `y_2016_04`

In [ ]:
timestamp = "2016-04-28"
timestamp_lags = ["2016-03-28", "2016-02-28", "2016-01-28", 
                  "2015-12-28", "2015-11-28", "2015-10-28", 
                  "2015-09-28", "2015-08-28", "2015-07-28", 
                  "2015-06-28", "2015-05-28", "2015-04-28"]

X_2016_04, y_2016_04 = extract_X_y_train(df_train, timestamp, timestamp_lags)

In [ ]:
X_2016_04.head()

In [ ]:
y_2016_04.head()

In [ ]:
X_2016_04.to_csv(os.path.join(OUT_DIR1, "X_2016_04.csv"), index=False)
y_2016_04.to_csv(os.path.join(OUT_DIR1, "y_2016_04.csv"), index=False)

## Extract `X_2016_05`, `y_2016_05`

In [ ]:
timestamp = "2016-05-28"
timestamp_lags = ["2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28",
                  "2015-12-28", "2015-11-28", "2015-10-28", 
                  "2015-09-28", "2015-08-28", "2015-07-28", 
                  "2015-06-28", "2015-05-28"]

X_2016_05, y_2016_05 = extract_X_y_train(df_train, timestamp, timestamp_lags)

In [ ]:
X_2016_05.head()

In [ ]:
y_2016_05.head()

In [ ]:
X_2016_05.to_csv(os.path.join(OUT_DIR1, "X_2016_05.csv"), index=False)
y_2016_05.to_csv(os.path.join(OUT_DIR1, "y_2016_05.csv"), index=False)

## Extract `X_2016_06`

In [ ]:
timestamp = "2016-06-28"
timestamp_lags = ["2016-05-28", "2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28",
                  "2015-12-28", "2015-11-28", "2015-10-28", "2015-09-28", 
                  "2015-08-28", "2015-07-28", "2015-06-28"]

X_2016_06 = extract_X_test(df_train, df_test, timestamp, timestamp_lags)

In [ ]:
X_2016_06.head()

In [ ]:
X_2016_06.to_csv(os.path.join(OUT_DIR1, "X_2016_06.csv"), index=False)